In [ ]:
import cv2
import requests
import numpy as np
from IPython.display import display, clear_output

# Set the URL for the camera stream
url = "http://192.168.0.179:80/"

# Set up the video stream
stream = requests.get(url, stream=True)
stream.raise_for_status()

# Initialize the face detection cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Loop through the video frames
for chunk in stream.iter_content(chunk_size=1024):
    # Decode the JPEG data
    img_array = bytes()
    img_array += chunk
    img = cv2.imdecode(np.frombuffer(img_array, dtype=np.uint8), cv2.IMREAD_COLOR)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Draw a rectangle around each face
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        
        # Send a signal to the ESP32 if a face is detected
        #requests.get('http://192.168.0.179/signal?300')

    # Display the image
    cv2.imshow('ESP32 Video Stream', cv2.imread(img))
    cv2.waitKey(1)
    
    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# Clean up
cv2.destroyAllWindows()
